In [2]:
#Побудувати нейронну мережу, аналогічну завданню 1 для датасета fashion mnist.
#Використати Keras Tuner підбору гіперпараметрів моделі.
#Порівняти результати завдання 1 і 2 (точність і функцію втрат).
!pip install keras-tuner -q

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from kerastuner.tuners import RandomSearch

fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print("x_train original shape", x_train.shape)
print("y_train original shape", y_train.shape)

x_train = x_train / 255.0
x_test = x_test / 255.0

def build_model(hp):
    model = keras.Sequential()
    model.add(Flatten(input_shape=(28, 28)))

    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                     min_value=32,
                                     max_value=512,
                                     step=32),
                        activation='relu',
                        kernel_regularizer=l2(0.001)))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_logs',
    project_name='fashion_mnist_classification'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

tuner.search(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[early_stopping, reduce_lr])

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

best_model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[early_stopping, reduce_lr])

test_loss, test_acc = best_model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Trial 10 Complete [00h 17m 22s]
val_accuracy: 0.8698999881744385

Best val_accuracy So Far: 0.8998000025749207
Total elapsed time: 03h 30m 54s


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 384)                 │         301,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 384)                 │           1,536 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 384)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 416)                 │         160,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 416)                 │           1,664 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 416)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 480)                 │         200,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 480)                 │           1,920 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 480)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           4,810 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 671,690 (2.56 MB)

 Trainable params: 669,130 (2.55 MB)

 Non-trainable params: 2,560 (10.00 KB)

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 35s 17ms/step - accuracy: 0.9039 - loss: 0.3408 - val_accuracy: 0.8908 - val_loss: 0.3969 - learning_rate: 1.0000e-04
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 33s 18ms/step - accuracy: 0.9039 - loss: 0.3470 - val_accuracy: 0.8822 - val_loss: 0.4135 - learning_rate: 1.0000e-04
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.8980 - loss: 0.3663 - val_accuracy: 0.8909 - val_loss: 0.4007 - learning_rate: 1.0000e-04
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9003 - loss: 0.3556 - val_accuracy: 0.8806 - val_loss: 0.4230 - learning_rate: 1.0000e-04
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 17ms/step - accuracy: 0.9089 - loss: 0.3381 - val_accuracy: 0.8957 - val_loss: 0.3839 - learning_rate: 5.0000e-05
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.9163 - loss: 0.3142 - val_accuracy: 0.8984 - val_loss: 0.3789 - learning_rate: 5.0000e-05
Epoch 7/50
1875/1875 ━━━━━━━━━━━━━━━━━━━

In [ ]:
#Висновки: Модель була створена за допомогою нейронних мереж у TensorFlow та Keras. Вхідні дані були згладжені за допомогою шару Flatten, щоб привести зображення до одновимірного вектору.
#Були додані кілька шарів Dense з активацією ReLU для вивчення нелінійних залежностей. Для кожного шару було додано BatchNormalization та Dropout для покращення узагальнення моделі і запобігання перенавчанню.
#Вихідний шар має 10 нейронів з активацією softmax, що дозволяє отримувати ймовірності для кожного класу.
#Використано Random Search для налаштування гіперпараметрів, таких як кількість шарів, кількість нейронів у кожному шарі, а також швидкість навчання.
#Найкраща модель була знайдена після 10 спроб, що дозволило оптимізувати точність моделі для завдання класифікації зображень.
#Використано регуляризацію L2 для зменшення впливу великих ваг і покращення узагальнення. Використано EarlyStopping і ReduceLROnPlateau для запобігання перенавчанню та адаптації швидкості навчання під час тренування.

#Найкраща модель продемонструвала точність на тестовому наборі даних, яка досягла приблизно 85-87%. Це є досить гарним результатом для базової нейронної мережі, що класифікує зображення одягу.
#Таким чином, проведена робота показала успішний підхід до класифікації зображень за допомогою глибоких нейронних мереж і дозволяє оцінити ефективність різних гіперпараметрів та методів регуляризації.

#Порівняння результатів завдання 1 та завдання 2:
#Дана модель досягла вищої точності на тестових даних (0.9056) порівняно з попередньою моделлю (0.8940).
#Це свідчить про те, що ця модель краще адаптувалася до задачі класифікації на даному наборі даних.
#Значення втрати в попередній мережі (0.3024) нижче, ніж у цій (0.3571), що може вказувати на краще узгодження цієї моделі щодо втрати.

#Посилання на гітхаб: https://github.com/nastiadzen/Machine-learning